## This notebook is about developing few-shot learning, model fine-tuning, and templates for guiding LLM outputs.

### Leveraging SFT(supervised fine tuning) to develop and deploy a AI Agent to generate sql query 

In [8]:
from datasets import load_dataset
from openai import OpenAI
import pandas as pd
from datasets import load_dataset
import pandas as pd
import numpy as np
from transformers import pipeline
from evaluate import load

### load training dataset spider from Hugging Face - with 100 in train and 10 in test

In [9]:
training_dataset, validation_dataset  = load_dataset('xlangai/spider', split=['train[0:100]', 'validation[0:10]'])
client = OpenAI()
llm_model = "gpt-4o-mini"

### Finetuning dataset preparation

In [188]:
def format_finetuning_dataset():
	system_prompt = "Spider knows SQL"
	dataset_dict = {"training": training_dataset, "validation": validation_dataset}

	def format_dataset(dataset_dict):
		formatted_data = []
		for dataset_name, dataset in dataset_dict.items():
			for row in dataset:
				question = row['question']
				query = row['query']
				message = {
					"messages": [
						{"role": "system", "content": system_prompt},
						{"role": "user", "content": question},
						{"role": "assistant", "content": query}
					]
				}
				formatted_data.append(message)
			formatted_df = pd.DataFrame(formatted_data)
			formatted_df.to_json(f"{dataset_name}_dataset.jsonl", orient='records', lines=True)

	format_dataset(dataset_dict)

### OpenAI wrapper

In [7]:
def call_openai(system_prompt,prompt, model=llm_model):
	chat_completion = client.chat.completions.create(
		messages=[
			{
				"role": "system",
				"content": system_prompt
			},
			{
				"role": "user",
				"content": prompt,
			}
		],
		model=model,
	)
	return chat_completion.choices[0].message.content


### Evaluating performance - exact match

In [256]:
def reformating_data(data):

	data['query'] = data['query'].apply(lambda x: re.sub(r'\s+', ' ',x))
	data['predicted_query'] = data['predicted_query'].apply(lambda x: re.sub(r'\s+', ' ',x))
	data['query'] = data['query'].str.lower()
	data['predicted_query'] = data['predicted_query'].str.lower()
	return data

def evaluate_dataset(model, is_finetuned=False):
	results = []
	for row in validation_dataset:
		if is_finetuned:
			system_prompt = "agent knows how to do SQL query"
			prompt = row['question']
		else:
			#few shot learning
			system_prompt = "Follow the user commands."
			prompt = f"Translate the text command to SQL. Only output the SQL.\n" \
					 f"Text: Return all data from the customers table  SQL: Select * from customers ###\n"\
					 f"Text: Return all data from the customers table for customers' age > 60  SQL: Select * from customers where age > 60 ###\n"\
					 f"Translate the text command to SQL. Only output the SQL.\n"\
					 f"Text: {row['question']} SQL: "
		result = call_openai(system_prompt,prompt, model)
		result = result.replace(";", "")
		results.append(result)

	validation_df = pd.DataFrame(validation_dataset)
	validation_df['predicted_query'] = results
	validation_df = reformating_data(validation_df)
	validation_df.to_csv(f"{model}_validation_accuracy_results.csv", index=False)
	accuracy = (validation_df['predicted_query'] == validation_df['query']).mean()
	print(f"Accuracy: {accuracy}")

### Upload dataset to OpenAI for SFT(supervised fine tuning)

In [193]:
def upload_dataset_to_openai():
	validation_file = client.files.create(
		file=open("validation_dataset.jsonl", "rb"),
		purpose="fine-tune"
	)
	training_file = client.files.create(
		file=open("training_dataset.jsonl", "rb"),
		purpose="fine-tune",
	)
	return training_file, validation_file

### OpenAI SFT(supervised fine tuning) 

In [120]:
def train_openai_model(training_file, validation_file):
	results = client.fine_tuning.jobs.create(
		training_file=training_file.id,
		validation_file=validation_file.id,
		model="gpt-3.5-turbo",
	)

### Performance few shot learning(w/o finetuning)

In [142]:
evaluate_dataset(model = llm_model)
format_finetuning_dataset()
evaluate_dataset(model=llm_model)

Accuracy: 0.0
Accuracy: 0.0


### Performance after finetuning

In [13]:
training_file, validation_file = upload_dataset_to_openai()
train_openai_model(training_file, validation_file)
evaluate_dataset(model="ft:gpt-3.5-turbo-0125:personal::AEyiOkSI", is_finetuned=True)
data = pd.read_csv("ft:gpt-3.5-turbo-0125:personal::AEyiOkSI_validation_accuracy_results.csv")
print(f" Accuracy of fine-tuned model: {np.sum(data['query'] == data['predicted_query'])/len(data) * 100:.2f}%")

 Accuracy of fine-tuned model: 70.00%


### SQL query generation using finetuned model

In [15]:
system_prompt = "Follow the user commands."
prompt = "Find the name of the employee who has the highest salary from table employee."
model = "ft:gpt-3.5-turbo-0125:personal::AEyiOkSI"
call_openai(system_prompt,prompt, model)

## Other evaluation metric

### As the output generated is string exact match of special characters like puctuation,spaces may not exactly match with the reference(truth).In that case following are a few evaluation metric to detect its functional accuracy or entailment etc. 

In [286]:
# %pip install evaluate

# %pip install bert_score
# %pip install rouge_score
bertscore = load("bertscore") # from paper https://arxiv.org/pdf/1904.09675.pdf
bleu = load("bleu") # from paper https://aclanthology.org/P02-1040.pdf
rouge = load("rouge") # from paper https://aclanthology.org/W04-1013.pdf
entailment_pipe = pipeline("text-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [299]:
def calculate_results(generated_results,references):
		bleu_scores = []
		rouge_scores = []
		bert_scores = []
		entailment_scores = []	
		for generated,reference in zip(generated_results,references):
			print("\nStatement: ", generated, "\nReference: ", reference)
			results = bleu.compute(predictions=generated, references=reference,max_order=2)
			print("Bleu", results)
			bleu_scores.append(results['bleu'])

			results = rouge.compute(predictions=generated, references=reference)
			print("Rouge", results)
			rouge_scores.append(results['rougeL'])
			
			results = bertscore.compute(predictions=generated, references=reference, lang="en")
			print("Bert Score", results)
			bert_scores.append(results['f1'][0])

			result = entailment_pipe({'text':generated, 'text_pair': reference})
			print("NLI Entailment", result)
			entailment_scores.append(result['score'])
		print(bleu_scores)		
		summary=pd.DataFrame()
		summary = pd.DataFrame({'blue':[np.mean(bleu_scores)]}) 
		summary['rouge']=np.mean(rouge_scores)
		summary['bert']=np.mean(bert_scores)
		summary['entailment']=np.mean(entailment_scores)
		return summary


generated_results = [[data['query'][i]] for i in range(len(data))]
references = [[data['predicted_query'][i]] for i in range(len(data))]
calculate_results(generated_results, references)


Statement:  ['select count(*) from singer'] 
Reference:  ['select count(*) from singer']
Bleu {'bleu': 1.0, 'precisions': [1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 7, 'reference_length': 7}
Rouge {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
Bert Score {'precision': [0.9999999403953552], 'recall': [0.9999999403953552], 'f1': [0.9999999403953552], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.42.0)'}
NLI Entailment {'label': 'entailment', 'score': 0.9697027802467346}

Statement:  ['select count(*) from singer'] 
Reference:  ['select count(*) from singer']
Bleu {'bleu': 1.0, 'precisions': [1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 7, 'reference_length': 7}
Rouge {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
Bert Score {'precision': [0.9999999403953552], 'recall': [0.9999999403953552], 'f1': [0.9999999403953552], 'hashcode': 'roberta-large_L17_no-idf_version=0.3

,blue,rouge,bert,entailment
0,0.861854,0.901587,0.981031,0.928754
